In [1]:
import geopandas as gpd
import line_orchard_detection as ld

In [ ]:
path = '../datasets/main_ortho1_42/'

ld.detect_lines_path(path)

In [2]:
path = '../VECTOR/Area/'

pontos = gpd.read_file(path + 'centroides_area.geojson')

falhas, linhas = ld.detect_lines_uni(pontos)

linhas.to_file(path + 'line_prediction.geojson', driver='GeoJSON')
# falhas.to_file(path + 'falhas_prediction.geojson', driver='GeoJSON')

In [ ]:
path = '../datasets/main_norte_42/'

pontos = gpd.read_file(path + 'centroids.geojson')
talhoes = gpd.read_file(path + 'talhoes.geojson').set_index('id')

falhas_completas, linhas_completas = ld.detect_lines_talhoes(pontos, talhoes)

for i, (falhas, linhas) in enumerate(zip(falhas_completas, linhas_completas)):
    falhas.to_file(path + 'gaps.gpkg', layer='{}'.format(i) , driver='GPKG')
    linhas.to_file(path + 'lines.gpkg', layer='{}'.format(i) , driver='GPKG')

In [ ]:
path = '../datasets/main_ortho1_42/'

pontos = gpd.read_file(path + 'centroids_analysis.geojson')

linhas, points = ld.is_orchard_line(pontos)
geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=pontos.crs)})
# geodf = extrapolar_linhas(geodf)
buffer = ld.get_buffer(geodf)
lines = ld.buffer_to_lines(buffer, points)
points = ld.mapping_lines(points, lines)
geodf.to_file(path + 'lines_prediction1.geojson', driver='GeoJSON')

linhas_novas = ld.snap_lines(points, lines)

for i in linhas_novas:
    linhas.append(i)

geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=points.crs)})
# geodf = extrapolar_linhas(geodf)
buffer = ld.get_buffer(geodf)
lines = ld.buffer_to_lines(buffer, points)
points = ld.mapping_lines(points, lines)
geodf.to_file(path + 'lines_prediction2.geojson', driver='GeoJSON')

linhas_novas = ld.snap_lines(points, lines)

for i in linhas_novas:
    linhas.append(i)

geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=points.crs)})
# geodf = extrapolar_linhas(geodf)
buffer = ld.get_buffer(geodf)
lines = ld.buffer_to_lines(buffer, points)
points = ld.mapping_lines(points, lines)
geodf.to_file(path + 'lines_prediction3.geojson', driver='GeoJSON')


# falhas.to_file(path + 'falhas_prediction.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import math

from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString, MultiLineString

In [ ]:
def get_angle(p1, p2):    
    
    y = p2.y-p1.y
    x = p2.x-p1.x
    
    angle = np.degrees(np.arctan2(y, x))
    
    return angle

def get_angle_vertex(p1, p_middle, p2):
    
    get_angle(p_middle, p1)
    get_angle(p_middle, p2)
    
    angle = get_angle(p_middle, p1) - get_angle(p_middle, p2)
    
    if angle < 0:
        angle += 360
    
    return angle

def is_line(p1, p_middle, p2, angle_threshold=10):
    
    angle = get_angle_vertex(p1, p_middle, p2)
    
    if round(abs(angle-180)) <= angle_threshold:
        return True
    else:
        return False

    
def getExtrapoledLine(p1,p2):
    'Creates a line extrapoled in p1->p2 direction'
    EXTRAPOL_RATIO = 2
    a = p1
    b = (p1[0]+EXTRAPOL_RATIO*(p2[0]-p1[0]), p1[1]+EXTRAPOL_RATIO*(p2[1]-p1[1]) )
    return b

def round_school(x):
    i, f = divmod(x, 1)
    return int(i + ((f >= 0.5) if (x > 0) else (f > 0.5)))

In [ ]:
def is_orchard_line(points, angle_threshold=18):
       
    points.set_index(points.index+1, inplace=True)
    points['id'] = points.index

    nA = np.array(list(points.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nA)
    dist, idx = btree.query(nA, k=13)

    points['pt_1'] = idx[:,1]+1
    points['pt_2'] = idx[:,2]+1
    # Não utilizados agora
    points['pt_3'] = idx[:,3]+1
    points['pt_4'] = idx[:,4]+1
    points['pt_5'] = idx[:,5]+1
    points['pt_6'] = idx[:,6]+1
    points['pt_7'] = idx[:,7]+1
    points['pt_8'] = idx[:,8]+1
    points['pt_9'] = idx[:,9]+1
    points['pt_10'] = idx[:,10]+1
    points['pt_11'] = idx[:,11]+1
    points['pt_12'] = idx[:,12]+1

    saida_linhas = []
    for i in points.index:

        first = points.loc[points.loc[i,'pt_1'],'geometry']
        second = points.loc[points.loc[i,'pt_2'],'geometry']

        if is_line(first, points.loc[i,'geometry'], second, angle_threshold=angle_threshold):
            saida_linhas.append(LineString([first, points.loc[i,'geometry'], second]))
#             points.at[i, 'middle'] = True
            
    return saida_linhas, points

def extrapolar_linhas(gdf):
    for i in gdf.index:
        coords = [[x,y] for x,y in zip(gdf.loc[i,'geometry'].xy[0], gdf.loc[i,'geometry'].xy[1])]
        coords.append(['first'])
        coords.append(['last'])
        coords[0], coords[1], coords[2], coords[3] = coords[-2], coords[0], coords[1], coords[2]

        coords[0] = getExtrapoledLine(coords[2], coords[1])
        coords[-1] = getExtrapoledLine(coords[2], coords[3])

        gdf.at[i, 'geometry'] = LineString(coords)
    return gdf

def get_buffer(geodf):
    buffer = geodf['geometry'].buffer(0.5).unary_union #.explode().reset_index(drop=True)
    df_buffer = gpd.GeoDataFrame({"geometry":buffer}, crs=geodf.crs)
    return df_buffer

def buffer_to_lines(buffer, points, linhas=None):
    
    if linhas is None:
        linhas = []
    
    points['buffer'] = -99
        
    for i in buffer.index:

        internos = points.geometry.within(buffer.loc[i, 'geometry'])
        select = points.loc[list([internos[internos].index][0])]

        x = np.array(list(select['geometry'].x))
        y = np.array(list(select['geometry'].y))

        points_selected = np.array([[x, y]for x, y in zip(list(select['geometry'].x), list(select['geometry'].y))])

        x_sorted = x.copy()
        x_sorted.sort()

        xy = []
        for z in x_sorted:
            xy.append(list(points_selected[points_selected[:,0] == z][0]))
            
        points.at[list([internos[internos].index][0]), 'buffer'] = int(i)

        linhas.append(LineString(xy))
    
    gdf_linhas = gpd.GeoDataFrame({"geometry":gpd.GeoSeries(linhas, crs=points.crs)})
    gdf_linhas['id'] = buffer.index
    gdf_linhas.set_index('id', inplace=True)
    return gdf_linhas
        
        
def mapping_lines(points, lines):
    
#     points['line'] = -99
    points['line'] = ''
    
    for i in lines.index:
        
        points_selected = points.loc[points['buffer'] == i]
        line = lines.loc[i,'geometry']
        
        xy_line = [[x,y] for x, y in zip(line.xy[0], line.xy[1])]
        
        first_point = Point(xy_line[0])
        nea_first_point = Point(xy_line[1])
        
        last_point = Point(xy_line[-1])
        nea_last_point = Point(xy_line[-2])
        
        id_first = points.loc[points['geometry'] == first_point].index[0]
        id_nea_first = points.loc[points['geometry'] == nea_first_point].index[0]
        
        id_last = points.loc[points['geometry'] == last_point].index[0]
        id_nea_last = points.loc[points['geometry'] == nea_last_point].index[0]
        
        points.at[id_first, 'line'] = 'first'
        points.at[id_last, 'line'] = 'last'
        
        if id_nea_first == id_nea_last:
            points.at[id_nea_first, 'line'] = 'nearest'
        else:
            points.at[id_nea_first, 'line'] = 'first_nearest'
            points.at[id_nea_last, 'line'] = 'last_nearest'
        
    return points

def select_bridge(possiveis_pontes):
  
    mapeamento = {}
    for ids, i in enumerate(possiveis_pontes):

        fracao = LineString([i[1], i[2]]).length / LineString([i[0], i[1]]).length
        angle = get_angle_vertex(i[0], i[1], i[2])
        
        mapeamento[ids] = {'fracao':fracao, 'angle':angle}
    
    df_select = pd.DataFrame(mapeamento).T
    df_select.set_index(pd.Index([x for x in range(df_select.shape[0])]), inplace=True)
    
    id_drop = df_select.loc[df_select['fracao'] > 3.5].index
    df_select = df_select.drop(index = id_drop, axis = 0)
    
    df_select = df_select.sort_values('fracao')


    if df_select.empty:
        return None
    elif df_select.shape[0] == 1:
        index = df_select.iloc[[0]].index[0]
    else:
        menor_1 = df_select.iloc[0]['fracao']
        menor_2 = df_select.iloc[1]['fracao']
        
        if (menor_2 / menor_1) <= 1.05:
            index = df_select.head(2).sort_values('angle').iloc[[0]].index[0]
        else:
            index = df_select.head(1).index[0]

    bridge = possiveis_pontes[index]

    return bridge

def snap_lines(points, lines):
    linhas_ponte = []
    for i in lines.index:

        selected = points.loc[points['buffer'] == i]

        for z in ['first', 'last']:

            point_1 = selected.loc[selected['line'] == z]

            if selected.shape[0] > 3:
                point_0_label = z + '_nearest'
            else:
                point_0_label = 'nearest'

            point_0 = selected.loc[selected['line'] == point_0_label]
            
            possiveis_pontes = []
            for y in range(1,13):
                num_point = point_1.iloc[0]['pt_{}'.format(y)]

                point_2 = points.loc[points['id']==num_point]

                if is_line(point_0.iloc[0].geometry, point_1.iloc[0].geometry, point_2.iloc[0].geometry, 45):
                                    
                    if point_2['line'].iloc[0] in ['first','last']:
                        
                        stop = False
                        pt_sufix = 1
                        while not stop:
                            point_aux = points.loc[points['id']==point_2['pt_{}'.format(pt_sufix)].iloc[0]]
                            
                            if 'nearest' in point_aux['line'].iloc[0]:
                                stop = True
                            pt_sufix += 1
                            
                        point_aux_geom = point_aux['geometry'].iloc[0]
                        
                        if is_line(point_1.iloc[0].geometry, point_2.iloc[0].geometry, point_aux_geom, 30):
                            possiveis_pontes.append([point_0.iloc[0].geometry, point_1.iloc[0].geometry, point_2.iloc[0].geometry])
                    elif point_2['buffer'].iloc[0] == -99:
                        possiveis_pontes.append([point_0.iloc[0].geometry, point_1.iloc[0].geometry, point_2.iloc[0].geometry])

            if len(possiveis_pontes) <= 0:
                pass
            else:
                linha_escolhida = select_bridge(possiveis_pontes)

                if linha_escolhida is not None:
                    linhas_ponte.append(LineString(linha_escolhida))
    return linhas_ponte

# Detectar falhas
def falhas_detect(linhas):
    falhas = gpd.GeoSeries()
    for i in linhas['geometry']:

        seg = list(map(LineString, zip(i.coords[:-1], i.coords[1:])))
        segments = gpd.GeoSeries(seg, crs=linhas.crs)

        dist = (segments.length.quantile(0.5) + segments.length.quantile(0.6))/2

        maiores = segments[segments.length > 1.15 * dist]

        if maiores.shape[0] == 0:
            continue
        else:
            target_lines = round((maiores.length / dist),2)
            index = target_lines.index
            split = list(target_lines)

            for idx, z in zip(index,split):
                z = round_school(z)
                slices = pd.Series([x/z for x in range(1,z)])
                for y in slices:
                    falha = maiores.loc[[idx]].interpolate(y, normalized=True)
                    falhas = pd.concat([falhas,falha], axis=0)
    return falhas

In [ ]:
'nearest' in 'last_nearest'

In [ ]:
selected = points.loc[points['buffer'] == 3]
point_1 = selected.loc[selected['line'] == 'last']
num_point = point_1.iloc[0]['pt_{}'.format(1)]
point_2 = points.loc[points['id']==num_point]
point_aux = points.loc[points['id']==point_2['pt_1'].iloc[0]]
point_aux['line'].iloc[0]

In [ ]:
def main_snap_lines(df_points):
    
    linhas, points = is_orchard_line(df_points)
    geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=df_points.crs)})
    geodf = extrapolar_linhas(geodf)
    buffer = get_buffer(geodf)
    lines = buffer_to_lines(buffer, points)
    points = mapping_lines(points, lines)
    
    points_new = gpd.GeoDataFrame()
    
    stop = True
    while stop:
        
        if not points_new.empty:
            points = points_new.copy()
        
        linhas_novas = snap_lines(points, lines)
        
        if len(linhas_novas) > 0:        
#             print(len(linhas))
#             print(len(linhas_novas))
            for i in linhas_novas:
                linhas.append(i)
            
            geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=points.crs)})
            geodf = extrapolar_linhas(geodf)
            buffer = get_buffer(geodf)
            lines = buffer_to_lines(buffer, points)
            points_new = mapping_lines(points, lines)
        else:
            stop = False
    falhas = falhas_detect(lines) 
    return falhas, lines

In [ ]:
path = '../datasets/main_ortho1_42/'

pontos = gpd.read_file(path + 'centroids_analysis.geojson')

linhas, points = is_orchard_line(pontos)
geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=pontos.crs)})
# geodf = extrapolar_linhas(geodf)
buffer = get_buffer(geodf)
lines = buffer_to_lines(buffer, points)
points = mapping_lines(points, lines)
geodf.to_file(path + 'lines_prediction1.geojson', driver='GeoJSON')

linhas_novas = snap_lines(points, lines)

for i in linhas_novas:
    linhas.append(i)

geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=points.crs)})
# geodf = extrapolar_linhas(geodf)
buffer = get_buffer(geodf)
lines = buffer_to_lines(buffer, points)
points = mapping_lines(points, lines)
geodf.to_file(path + 'lines_prediction2.geojson', driver='GeoJSON')

linhas_novas = snap_lines(points, lines)

for i in linhas_novas:
    linhas.append(i)

geodf = gpd.GeoDataFrame({'geometry':gpd.GeoSeries(linhas, crs=points.crs)})
# geodf = extrapolar_linhas(geodf)
buffer = get_buffer(geodf)
lines = buffer_to_lines(buffer, points)
points = mapping_lines(points, lines)
geodf.to_file(path + 'lines_prediction3.geojson', driver='GeoJSON')


# falhas.to_file(path + 'falhas_prediction.geojson', driver='GeoJSON')

In [ ]:
pontos = gpd.read_file('../VECTOR/line_points_all_norte.geojson')

falhas, lines = main_snap_lines(pontos)

In [ ]:
lines.to_file('../VECTOR/line_prediction_norte.geojson', driver='GeoJSON')
falhas.to_file('../VECTOR/falhas_prediction_norte.geojson', driver='GeoJSON')

In [ ]:
get_centroids(canopy)

In [ ]:
# Detectar falhas
falhas = gpd.GeoSeries()
for i in saida_linhas_nova:
    
    seg = list(map(LineString, zip(i.coords[:-1], i.coords[1:])))
    segments = gpd.GeoSeries(seg, crs=points.crs)
    
    dist = (segments.length.quantile(0.6) + segments.length.quantile(0.7))/2
    
    maiores = segments[segments.length > 1.35 * dist]

    if maiores.shape[0] == 0:
        continue
    else:
        target_lines = round((maiores.length / dist),2)
        index = target_lines.index
        split = list(target_lines)
        
        for idx, z in zip(index,split):
            z = round_school(z)
            slices = pd.Series([x/z for x in range(1,z)])
            for y in slices:
                falha = maiores.loc[[idx]].interpolate(y, normalized=True)
                falhas = pd.concat([falhas,falha], axis=0)
                
falhas.to_file('../VECTOR/falhas_prediction_sul.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd

In [ ]:
pols = gpd.read_file('../datasets/main_norte_42_2/canopy_detection_result.geojson')
pols

In [ ]:
pols.explode()